In [1]:
import os
import django
import pandas as pd
import numpy as np
import datetime
from django.db import transaction
from django.utils import timezone
from pathlib import Path
from django.db.models import Max

# os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend.settings")
# os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
# django.setup()

# from mainApp.models import Field, FieldByYear, Course, Semester, Faculty
import sqlite3

In [48]:
faculty_df = pd.read_csv("sylabus_data/faculty_data_2025_1_stopien.csv", sep=';', index_col=0)
course_df = pd.read_csv("sylabus_data/course_data_2025_1_stopien.csv", sep=';', index_col=0)
field_df = pd.read_csv("sylabus_data/field_data_2025_1_stopien.csv", sep=';', index_col=0)

In [49]:
faculty_df.head()

,name
0,Wydział Inżynierii Lądowej i Gospodarki Zasobami
1,Wydział Inżynierii Metali i Informatyki Przemy...
2,"Wydział Elektrotechniki, Automatyki, Informaty..."
3,"Wydział Informatyki, Elektroniki i Telekomunik..."
4,Wydział Inżynierii Mechanicznej i Robotyki


In [50]:
field_df.head()

,faculty_id,name,kind,type,description,specialization
0,0,Budownictwo,Studia inżynierskie I stopnia,Stacjonarne,Kierunek studiów Budownictwo należy do obszaru...,NaN
1,0,Budownictwo,Studia inżynierskie I stopnia,Niestacjonarne,Kierunek studiów Budownictwo należy do obszaru...,NaN
2,0,Inżynieria Górnicza,Studia inżynierskie I stopnia,Stacjonarne,Kierunek studiów Inżynieria Górnicza należy do...,NaN
3,0,Inżynieria Górnicza,Studia inżynierskie I stopnia,Niestacjonarne,Kierunek studiów Inżynieria Górnicza należy do...,NaN
4,0,Inżynieria i Zarządzanie Procesami Przemysłowymi,Studia inżynierskie I stopnia,Stacjonarne,Kierunek studiów Inżynieria i Zarządzanie Proc...,NaN


In [52]:
course_df.head()

,field_id,name,semester,ects,test_type,lecturer,description
0,0,Geologia,1,3.0,Zaliczenie,Katarzyna Cyran,Zajęcia prowadzone w trybie mieszanym. Deforma...
1,0,Geometria wykreślna,1,2.0,Zaliczenie,Krzysztof Pałac,Ćwiczenia polegają na samodzielnym wykonywaniu...
2,0,Grafika inżynierska i rysunek techniczny,1,3.0,Zaliczenie,Sebastian Olesiak,Rysunek techniczny z zastosowaniem tradycyjnyc...
3,0,Geodezja,1,3.0,Zaliczenie,Michał Strach,1. Podstawowe pojęcia geodezyjne. Organizacja ...
4,0,Fizyka I,1,3.0,Zaliczenie,Michał Ślęzak,Wykład: Głównym celem wykładów jest jakościowe...


### Mapowanie nazw wydziałów (id) na nowe (id) w tabeli field w przypadku różnic w bazie i pliku

In [53]:
cnx = sqlite3.connect('db.sqlite3')
sql_faculty_df = pd.read_sql_query("SELECT * FROM mainApp_faculty", cnx, index_col='id')

In [54]:
faculty_map = dict(zip(faculty_df.index.values, faculty_df.name))

In [55]:
sql_faculty_df

,name,building_id
id,,
202,Wydział Inżynierii Lądowej i Gospodarki Zasobami,1
203,Wydział Inżynierii Metali i Informatyki Przemy...,1
204,"Wydział Elektrotechniki, Automatyki, Informaty...",1
205,"Wydział Informatyki, Elektroniki i Telekomunik...",1
206,Wydział Inżynierii Mechanicznej i Robotyki,1
207,"Wydział Geologii, Geofizyki i Ochrony Środowiska",1
208,Wydział Geodezji Górniczej i Inżynierii Środow...,1
209,Wydział Inżynierii Materiałowej i Ceramiki,1
210,Wydział Odlewnictwa,1


In [56]:
sql_faculty_df = sql_faculty_df[['name', 'building_id']].copy()

# --- KROK 2: Przygotuj faculty_df ---
# Zresetuj index (bo masz je od 0) i zostaw tylko 'name'
faculty_df = faculty_df[['name']].copy()
faculty_df = faculty_df.drop_duplicates(subset=['name'])

# --- KROK 3: Znajdź nazwy, których jeszcze nie ma w sql_faculty_df ---
existing_names = sql_faculty_df['name'].unique()
new_faculty_df = faculty_df[~faculty_df['name'].isin(existing_names)].copy()

# --- KROK 4: Nadaj nowe ID ---
if not sql_faculty_df.empty:
    next_id = sql_faculty_df.index.max() + 1
else:
    next_id = 1

new_faculty_df['building_id'] = 1  # lub domyślna wartość
new_faculty_df.index = np.arange(next_id, next_id + len(new_faculty_df))

# --- KROK 5: Połącz wszystko ---
final_faculty_df = pd.concat([sql_faculty_df, new_faculty_df])

# --- KROK 6: Zabezpiecz typ danych ---
final_faculty_df['building_id'] = final_faculty_df['building_id'].astype('int16')

# --- KROK 7: Posortuj po id (opcjonalnie) ---
final_faculty_df = final_faculty_df.sort_index()

In [57]:
faculty_df = final_faculty_df

In [58]:
faculty_df

,name,building_id
202,Wydział Inżynierii Lądowej i Gospodarki Zasobami,1
203,Wydział Inżynierii Metali i Informatyki Przemy...,1
204,"Wydział Elektrotechniki, Automatyki, Informaty...",1
205,"Wydział Informatyki, Elektroniki i Telekomunik...",1
206,Wydział Inżynierii Mechanicznej i Robotyki,1
207,"Wydział Geologii, Geofizyki i Ochrony Środowiska",1
208,Wydział Geodezji Górniczej i Inżynierii Środow...,1
209,Wydział Inżynierii Materiałowej i Ceramiki,1
210,Wydział Odlewnictwa,1
211,Wydział Metali Nieżelaznych,1


In [59]:
faculty_map

{np.int64(0): 'Wydział Inżynierii Lądowej i Gospodarki Zasobami',
 np.int64(1): 'Wydział Inżynierii Metali i Informatyki Przemysłowej',
 np.int64(2): 'Wydział Elektrotechniki, Automatyki, Informatyki i Inżynierii Biomedycznej',
 np.int64(3): 'Wydział Informatyki, Elektroniki i Telekomunikacji',
 np.int64(4): 'Wydział Inżynierii Mechanicznej i Robotyki',
 np.int64(5): 'Wydział Geologii, Geofizyki i Ochrony Środowiska',
 np.int64(6): 'Wydział Geodezji Górniczej i Inżynierii Środowiska',
 np.int64(7): 'Wydział Inżynierii Materiałowej i Ceramiki',
 np.int64(8): 'Wydział Odlewnictwa',
 np.int64(9): 'Wydział Metali Nieżelaznych',
 np.int64(10): 'Wydział Wiertnictwa, Nafty i Gazu',
 np.int64(11): 'Wydział Zarządzania',
 np.int64(12): 'Wydział Energetyki i Paliw',
 np.int64(13): 'Wydział Fizyki i Informatyki Stosowanej',
 np.int64(14): 'Wydział Matematyki Stosowanej',
 np.int64(15): 'Wydział Humanistyczny',
 np.int64(16): 'Wydział Informatyki',
 np.int64(17): 'Wydział Technologii Kosmicznych'}

In [60]:
new_faculty_map = {}
for key, val in faculty_map.items():
    new_faculty_map[key] = faculty_df.index[faculty_df['name'] == val].tolist()[0]

In [61]:
field_df['faculty_id'] = field_df['faculty_id'].replace(new_faculty_map)
field_df['specialization'] = field_df['specialization'].fillna('')
field_df['formula'] = ''

field_level_map = {'Studia inżynierskie I stopnia': 'I stopień', 'Studia licencjackie I stopnia': 'I stopień'}
field_df = field_df.rename(columns={'kind': 'level'})
field_df['level'] = field_df['level'].replace(field_level_map)
field_df['type'] = field_df['type'].str.lower()

field_df = field_df.replace('Edukacja Techniczno - Informatyczna', 'Edukacja Techniczno-Informatyczna')

field_df.head()

,faculty_id,name,level,type,description,specialization,formula
0,202,Budownictwo,I stopień,stacjonarne,Kierunek studiów Budownictwo należy do obszaru...,,
1,202,Budownictwo,I stopień,niestacjonarne,Kierunek studiów Budownictwo należy do obszaru...,,
2,202,Inżynieria Górnicza,I stopień,stacjonarne,Kierunek studiów Inżynieria Górnicza należy do...,,
3,202,Inżynieria Górnicza,I stopień,niestacjonarne,Kierunek studiów Inżynieria Górnicza należy do...,,
4,202,Inżynieria i Zarządzanie Procesami Przemysłowymi,I stopień,stacjonarne,Kierunek studiów Inżynieria i Zarządzanie Proc...,,


### Usuwanie/zamiana nazw kierunków

In [64]:
zamiany = {'Informatyka (kierunek wspólny - WEAIiIB)': 'Informatyka', 
           'Informatyka (kierunek wspólny - WI)': 'Informatyka',
           'Mechatronic Engineering with English as instruction language': 'Mechatronic Engineering',
           'Technologia Chemiczna (kierunek wspólny - WIMiC)': 'Technologia Chemiczna',
           'Technologia Chemiczna (kierunek wspólny - WEiP)': 'Technologia Chemiczna',
           'Technologie Przemysłu 4.0 (kierunek wspólny - WEAIiIB, WO)': 'Technologie Przemysłu 4.0'}
field_df['name'] = field_df['name'].replace(zamiany)

In [65]:
field_df['is_active'] = 1

In [66]:
field_df.shape

(92, 8)

### Mapowanie nazw kierunków (id) na nowe (id) w tabeli course w przypadku różnic w bazie i pliku

In [68]:
sql_field_df = pd.read_sql_query("SELECT * FROM mainApp_field", cnx, index_col='id')
sql_field_df = sql_field_df[sql_field_df['description'].notna()]    # usunięcie NULL
sql_field_df = sql_field_df.drop_duplicates(['name', 'type'])
sql_field_df[['description', 'specialization']] = sql_field_df[['description', 'specialization']].fillna('')
sql_field_df['faculty_id'] = ''
sql_field_df['level'] = ''
sql_field_df

,name,formula,description,specialization,type,level,isActive,faculty_id
id,,,,,,,,
1475,Budownictwo,2*M+3*G1+G2,Kierunek studiów Budownictwo należy do obszaru...,,stacjonarne,,1,
1476,Inżynieria Górnicza,2*M+3*G1+G2,Kierunek studiów Inżynieria Górnicza należy do...,,stacjonarne,,1,
1477,Inżynieria Kształtowania Środowiska,2*M+3*G1+G2,Studenci otrzymują wiedzę ogólną z zakresu inż...,,stacjonarne,,1,
1478,Inżynieria i Zarządzanie Procesami Przemysłowymi,2*M+3*G1+G2,Kierunek studiów Inżynieria i Zarządzanie Proc...,,stacjonarne,,1,
1479,Rewitalizacja Terenów Zdegradowanych,2*M+3*G1+G2,Program studiów I stopnia na kierunku Rewitali...,,stacjonarne,,1,
...,...,...,...,...,...,...,...,...
1567,Cyfryzacja Przemysłu,2*M+3*G1+G2,Transformacja cyfrowa ma ogromny wpływ na gosp...,,stacjonarne,,1,
1568,Recykling i Metalurgia,2*M+3*G1+G2,Studia inżynierskie niestacjonarne na kierunku...,,niestacjonarne,,1,
1569,Geotechnologie Otworowe,2*M+3*G1+G2,Opis kierunku studiów Geotechnologie Otworowe ...,,stacjonarne,,1,


In [69]:
field_df

,faculty_id,name,level,type,description,specialization,formula,is_active
0,202,Budownictwo,I stopień,stacjonarne,Kierunek studiów Budownictwo należy do obszaru...,,,1
1,202,Budownictwo,I stopień,niestacjonarne,Kierunek studiów Budownictwo należy do obszaru...,,,1
2,202,Inżynieria Górnicza,I stopień,stacjonarne,Kierunek studiów Inżynieria Górnicza należy do...,,,1
3,202,Inżynieria Górnicza,I stopień,niestacjonarne,Kierunek studiów Inżynieria Górnicza należy do...,,,1
4,202,Inżynieria i Zarządzanie Procesami Przemysłowymi,I stopień,stacjonarne,Kierunek studiów Inżynieria i Zarządzanie Proc...,,,1
...,...,...,...,...,...,...,...,...
87,217,Kulturoznawstwo,I stopień,stacjonarne,Kierunek Kulturoznawstwo łączy wiedzę humanist...,,,1
88,217,Socjologia,I stopień,stacjonarne,Celem oferty edukacyjnej proponowanej w ramach...,,,1
89,218,Informatyka - Zarządzanie Bezpieczeństwem Info...,I stopień,stacjonarne,Celem kształcenia na kierunku Informatyka – Za...,,,1
90,218,Informatyka,I stopień,stacjonarne,Kształcenie na studiach I stopnia realizuje ko...,,,1


In [70]:
# Zakładamy: sql_field_df oraz field_df są już wczytane

# 1. Przywróć kolumnę 'id' w sql_field_df (jeśli potrzebna później)
sql_field_df = sql_field_df.reset_index()

# 2. Merge tylko po wspólnych 'name' i 'type'
merged = pd.merge(
    sql_field_df,
    field_df[['name', 'type', 'description', 'specialization', 'faculty_id', 'level']],
    on=['name', 'type'],
    how='left',
    suffixes=('', '_new')
)

# 3. Uzupełnienie braków tylko jeśli oryginalna wartość to ''
for col in ['description', 'specialization', 'faculty_id', 'level']:
    merged[col] = merged[col].mask(merged[col] == '', merged[f'{col}_new'])

# 4. Usuń tymczasowe *_new kolumny
merged = merged.drop(columns=[f'{col}_new' for col in ['description', 'specialization', 'faculty_id', 'level']])

# 5. Przywróć 'id' jako indeks
merged = merged.set_index('id')

# 6. Dodaj nowe wiersze z field_df, których nie ma w sql_field_df
#    (czyli brakujące pary name+type)
existing_keys = set(zip(sql_field_df['name'], sql_field_df['type']))
new_rows = field_df[~field_df[['name', 'type']].apply(tuple, axis=1).isin(existing_keys)].copy()

# Dodaj nowe ID
last_id = sql_field_df['id'].max()
new_ids = range(last_id + 1, last_id + 1 + len(new_rows))
new_rows['id'] = new_ids

# Dodaj kolumny których nie ma w field_df (np. formula)
if 'formula' not in new_rows.columns:
    new_rows['formula'] = None

# Przywróć kolejność kolumn zgodną z merged
new_rows = new_rows[['id', 'name', 'formula', 'description', 'specialization', 'type', 'faculty_id', 'level']]
new_rows = new_rows.set_index('id')

# 7. Połącz wszystko razem
final_df = pd.concat([merged, new_rows])

In [21]:
final_df[final_df['name'].str.startswith('Technologia Chemiczna')]

,name,formula,description,specialization,type,level,isActive,faculty_id
id,,,,,,,,
1513,Technologia Chemiczna,2*M+3*G1+G2,Podjęcie studiów na kierunku Technologia chemi...,,stacjonarne,I stopień,1.0,209.0
1513,Technologia Chemiczna,2*M+3*G1+G2,Podjęcie studiów na kierunku Technologia chemi...,,stacjonarne,I stopień,1.0,214.0


In [73]:
final_df['formula'] = '2*M+3*G1+G2'
final_df['isActive'] = 1

In [74]:
final_df

,name,formula,description,specialization,type,level,isActive,faculty_id
id,,,,,,,,
1475,Budownictwo,2*M+3*G1+G2,Kierunek studiów Budownictwo należy do obszaru...,,stacjonarne,I stopień,1,202.0
1476,Inżynieria Górnicza,2*M+3*G1+G2,Kierunek studiów Inżynieria Górnicza należy do...,,stacjonarne,I stopień,1,202.0
1477,Inżynieria Kształtowania Środowiska,2*M+3*G1+G2,Studenci otrzymują wiedzę ogólną z zakresu inż...,,stacjonarne,I stopień,1,202.0
1478,Inżynieria i Zarządzanie Procesami Przemysłowymi,2*M+3*G1+G2,Kierunek studiów Inżynieria i Zarządzanie Proc...,,stacjonarne,I stopień,1,202.0
1479,Rewitalizacja Terenów Zdegradowanych,2*M+3*G1+G2,Program studiów I stopnia na kierunku Rewitali...,,stacjonarne,I stopień,1,202.0
...,...,...,...,...,...,...,...,...
1584,Geotechnologie Otworowe,2*M+3*G1+G2,Opis kierunku studiów Geotechnologie Otworowe ...,,niestacjonarne,I stopień,1,212
1585,Inżynieria Naftowa i Gazownicza,2*M+3*G1+G2,Plany inwestycyjne czołowych krajowych przedsi...,,niestacjonarne,I stopień,1,212
1586,Informatyka i Ekonometria,2*M+3*G1+G2,Informatyka i Ekonometria na Wydziale Zarządza...,,niestacjonarne,I stopień,1,213


In [76]:
# Przywróć indeks jako kolumnę w field_df i final_df
field_df = field_df.reset_index()  # teraz 'id' z field_df to kolumna
final_df = final_df.reset_index()  # teraz 'id' z final_df to kolumna

# Utwórz klucz dopasowania: (name, type, level)
field_df['match_key'] = list(zip(field_df['name'], field_df['type'], field_df['level']))
final_df['match_key'] = list(zip(final_df['name'], final_df['type'], final_df['level']))

# Połącz stare ID z nowym ID przez wspólne klucze
merge_keys = pd.merge(
    field_df[['index', 'match_key']],      # 'index' = stare ID
    final_df[['id', 'match_key']],         # 'id' = nowe ID
    on='match_key',
    how='inner'
)

# Stwórz słownik: stare_id -> nowe_id
field_id_map = dict(zip(merge_keys['index'], merge_keys['id']))
field_id_map

{0: 1475,
 1: 1573,
 2: 1476,
 3: 1555,
 4: 1478,
 5: 1574,
 6: 1477,
 7: 1575,
 8: 1479,
 9: 1480,
 10: 1481,
 11: 1576,
 12: 1540,
 13: 1577,
 14: 1483,
 15: 1541,
 16: 1578,
 17: 1485,
 18: 1489,
 19: 1486,
 20: 1579,
 21: 1493,
 22: 1538,
 23: 1564,
 24: 1487,
 25: 1488,
 26: 1542,
 27: 1495,
 28: 1492,
 29: 1490,
 30: 1491,
 31: 1580,
 32: 1566,
 33: 1494,
 34: 1496,
 35: 1581,
 36: 1567,
 37: 1497,
 38: 1498,
 39: 1499,
 40: 1546,
 41: 1501,
 42: 1582,
 43: 1500,
 44: 1502,
 45: 1503,
 46: 1504,
 47: 1505,
 48: 1506,
 49: 1508,
 50: 1507,
 51: 1509,
 52: 1583,
 53: 1510,
 54: 1511,
 55: 1512,
 56: 1482,
 57: 1565,
 58: 1513,
 59: 1514,
 60: 1516,
 61: 1543,
 62: 1517,
 63: 1518,
 64: 1568,
 65: 1569,
 66: 1584,
 67: 1521,
 68: 1585,
 69: 1523,
 70: 1586,
 71: 1524,
 72: 1587,
 73: 1525,
 74: 1588,
 75: 1526,
 76: 1527,
 77: 1528,
 78: 1513,
 79: 1544,
 80: 1529,
 81: 1530,
 82: 1531,
 83: 1532,
 84: 1539,
 85: 1533,
 86: 1534,
 87: 1535,
 88: 1536,
 89: 1571,
 90: 1493,
 91: 1572

In [77]:
# Podmień przestarzałe field_id na nowe
course_df['field_id'] = course_df['field_id'].map(field_id_map)

## Podsumowanie

In [79]:
faculty_df

,name,building_id
202,Wydział Inżynierii Lądowej i Gospodarki Zasobami,1
203,Wydział Inżynierii Metali i Informatyki Przemy...,1
204,"Wydział Elektrotechniki, Automatyki, Informaty...",1
205,"Wydział Informatyki, Elektroniki i Telekomunik...",1
206,Wydział Inżynierii Mechanicznej i Robotyki,1
207,"Wydział Geologii, Geofizyki i Ochrony Środowiska",1
208,Wydział Geodezji Górniczej i Inżynierii Środow...,1
209,Wydział Inżynierii Materiałowej i Ceramiki,1
210,Wydział Odlewnictwa,1
211,Wydział Metali Nieżelaznych,1


In [80]:
final_df

,id,name,formula,description,specialization,type,level,isActive,faculty_id,match_key
0,1475,Budownictwo,2*M+3*G1+G2,Kierunek studiów Budownictwo należy do obszaru...,,stacjonarne,I stopień,1,202.0,"(Budownictwo, stacjonarne, I stopień)"
1,1476,Inżynieria Górnicza,2*M+3*G1+G2,Kierunek studiów Inżynieria Górnicza należy do...,,stacjonarne,I stopień,1,202.0,"(Inżynieria Górnicza, stacjonarne, I stopień)"
2,1477,Inżynieria Kształtowania Środowiska,2*M+3*G1+G2,Studenci otrzymują wiedzę ogólną z zakresu inż...,,stacjonarne,I stopień,1,202.0,"(Inżynieria Kształtowania Środowiska, stacjona..."
3,1478,Inżynieria i Zarządzanie Procesami Przemysłowymi,2*M+3*G1+G2,Kierunek studiów Inżynieria i Zarządzanie Proc...,,stacjonarne,I stopień,1,202.0,(Inżynieria i Zarządzanie Procesami Przemysłow...
4,1479,Rewitalizacja Terenów Zdegradowanych,2*M+3*G1+G2,Program studiów I stopnia na kierunku Rewitali...,,stacjonarne,I stopień,1,202.0,"(Rewitalizacja Terenów Zdegradowanych, stacjon..."
...,...,...,...,...,...,...,...,...,...,...
94,1584,Geotechnologie Otworowe,2*M+3*G1+G2,Opis kierunku studiów Geotechnologie Otworowe ...,,niestacjonarne,I stopień,1,212,"(Geotechnologie Otworowe, niestacjonarne, I st..."
95,1585,Inżynieria Naftowa i Gazownicza,2*M+3*G1+G2,Plany inwestycyjne czołowych krajowych przedsi...,,niestacjonarne,I stopień,1,212,"(Inżynieria Naftowa i Gazownicza, niestacjonar..."
96,1586,Informatyka i Ekonometria,2*M+3*G1+G2,Informatyka i Ekonometria na Wydziale Zarządza...,,niestacjonarne,I stopień,1,213,"(Informatyka i Ekonometria, niestacjonarne, I ..."
97,1587,Zarządzanie,2*M+3*G1+G2,Zarządzanie na Wydziale Zarządzania Akademii G...,,niestacjonarne,I stopień,1,213,"(Zarządzanie, niestacjonarne, I stopień)"


In [81]:
course_df

,field_id,name,semester,ects,test_type,lecturer,description
0,1475,Geologia,1,3.0,Zaliczenie,Katarzyna Cyran,Zajęcia prowadzone w trybie mieszanym. Deforma...
1,1475,Geometria wykreślna,1,2.0,Zaliczenie,Krzysztof Pałac,Ćwiczenia polegają na samodzielnym wykonywaniu...
2,1475,Grafika inżynierska i rysunek techniczny,1,3.0,Zaliczenie,Sebastian Olesiak,Rysunek techniczny z zastosowaniem tradycyjnyc...
3,1475,Geodezja,1,3.0,Zaliczenie,Michał Strach,1. Podstawowe pojęcia geodezyjne. Organizacja ...
4,1475,Fizyka I,1,3.0,Zaliczenie,Michał Ślęzak,Wykład: Głównym celem wykładów jest jakościowe...
...,...,...,...,...,...,...,...
7310,1572,Theory of Fun in Game,6,4.0,Completing the classes,Jowita Guja,Theoretical aspects of pleasure-oriented game ...
7311,1572,Pracownia problemowa - ścieżka graficzna (arty...,6,4.0,Zaliczenie,Alicja Panasiewicz,W ramach pracowni prolemowej artystycznej stud...
7312,1572,Pracownia problemowa - ścieżka producencka,6,4.0,Zaliczenie,Jowita Guja,"Konsultacje, brainstorming, wybór tematów proj..."
7313,1572,Pracownia problemowa - ścieżka programistyczna,6,4.0,Zaliczenie,Krzysztof Boryczko,W ramach pracowni problemowej student realizuj...


# Uzupełnienie braków i stworzenie tabel odwołań

In [82]:
final_df[final_df['name'].str.startswith('Technologia Chemiczna')]

,id,name,formula,description,specialization,type,level,isActive,faculty_id,match_key
39,1513,Technologia Chemiczna,2*M+3*G1+G2,Podjęcie studiów na kierunku Technologia chemi...,,stacjonarne,I stopień,1,209.0,"(Technologia Chemiczna, stacjonarne, I stopień)"
40,1513,Technologia Chemiczna,2*M+3*G1+G2,Podjęcie studiów na kierunku Technologia chemi...,,stacjonarne,I stopień,1,214.0,"(Technologia Chemiczna, stacjonarne, I stopień)"


In [83]:
final_df.loc[final_df['formula'] == '', 'formula'] = '2*M+3*G1+G2'
final_df = final_df.drop(columns='match_key')
final_df = final_df.fillna('')
final_df['faculty_id'] = final_df['faculty_id'].replace('', np.nan)
final_df['faculty_id'] = final_df['faculty_id'].astype('Int64').astype(str)
final_df['faculty_id'] = final_df['faculty_id'].replace('<NA>', '')

C:\Users\szyme\AppData\Local\Temp\ipykernel_29220\877014057.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df['faculty_id'] = final_df['faculty_id'].replace('', np.nan)


In [85]:
# Przygotowanie danych
field_faculty_df = pd.DataFrame(final_df[['id', 'faculty_id']])
field_faculty_df = field_faculty_df.rename(columns={'id': 'field_id'})

sql_field_faculty_df = pd.read_sql_query("SELECT * FROM mainApp_field_faculty", cnx, index_col='id')
sql_field_faculty_df = sql_field_faculty_df.reset_index()

# Tworzenie mapy tylko dla wartości innych niż 29 i 30
field_faculty_map = dict(zip(sql_field_faculty_df['field_id'], sql_field_faculty_df['faculty_id']))
field_faculty_map = {k: v for k, v in field_faculty_map.items() if v not in [29, 30]}

In [86]:
field_faculty_df[field_faculty_df['field_id']==1513]

,field_id,faculty_id
39,1513,209
40,1513,214


In [87]:
final_df = final_df.drop_duplicates(subset=['name', 'description', 'type', 'level'])

In [89]:
final_df[final_df['name'].str.startswith('Zarządzanie')]

,id,name,formula,description,specialization,type,level,isActive,faculty_id
51,1524,Zarządzanie,2*M+3*G1+G2,Zarządzanie na Wydziale Zarządzania Akademii G...,,stacjonarne,I stopień,1,213
52,1525,Zarządzanie i Inżynieria Produkcji,2*M+3*G1+G2,Zarządzanie i Inżynieria Produkcji na Wydziale...,,stacjonarne,I stopień,1,213
97,1587,Zarządzanie,2*M+3*G1+G2,Zarządzanie na Wydziale Zarządzania Akademii G...,,niestacjonarne,I stopień,1,213
98,1588,Zarządzanie i Inżynieria Produkcji,2*M+3*G1+G2,Zarządzanie i Inżynieria Produkcji na Wydziale...,,niestacjonarne,I stopień,1,213


In [35]:
# final_df.iloc[100:130,:]

In [90]:
course_df['test_type'] = course_df['test_type'].str.lower()
course_test_type_map = {'completing the classes': 'zaliczenie', 'exam':'egzamin'}
course_df['test_type'] = course_df['test_type'].replace(course_test_type_map)
course_df = course_df.dropna(subset=['name', 'ects', 'test_type'])
course_df = course_df.reset_index()

In [91]:
course_df[course_df['field_id']==1342]

,index,field_id,name,semester,ects,test_type,lecturer,description


In [38]:
# # Przygotowanie danych
# field_faculty_df = pd.DataFrame(final_df[['id', 'faculty_id']])
# field_faculty_df = field_faculty_df.rename(columns={'id': 'field_id'})

# sql_field_faculty_df = pd.read_sql_query("SELECT * FROM mainApp_field_faculty", cnx, index_col='id')
# sql_field_faculty_df = sql_field_faculty_df.reset_index()

# # Tworzenie mapy tylko dla wartości innych niż 29 i 30
# field_faculty_map = dict(zip(sql_field_faculty_df['field_id'], sql_field_faculty_df['faculty_id']))
# field_faculty_map = {k: v for k, v in field_faculty_map.items() if v not in [29, 30]}


# field_faculty_df

In [92]:
field_faculty_df[field_faculty_df['field_id']==1513]

,field_id,faculty_id
39,1513,209
40,1513,214


In [93]:
course_field_df = pd.DataFrame(course_df[['index', 'field_id']])
course_field_df = course_field_df.rename(columns={'index': 'course_id'})

course_field_df

,course_id,field_id
0,0,1475
1,1,1475
2,2,1475
3,3,1475
4,4,1475
...,...,...
7307,7310,1572
7308,7311,1572
7309,7312,1572
7310,7313,1572


# Wrzucenie do bazy danych

## faculty

In [41]:
faculty_df = faculty_df.reset_index()  # upewnij się, że 'index' jest kolumną
faculty_df = faculty_df.rename(columns={'index': 'id'})  # dopasuj do bazy

# Połączenie z bazą danych
conn = sqlite3.connect('db.sqlite3')

cursor = conn.cursor()
cursor.execute("DELETE FROM mainApp_faculty")

# Wstawianie danych — ignoruj, jeśli dany 'id' już istnieje
cursor.executemany(
    """
    INSERT OR IGNORE INTO mainApp_faculty (id, name, building_id)
    VALUES (?, ?, ?)
    """,
    faculty_df[['id', 'name', 'building_id']].values.tolist()
)

conn.commit()
conn.close()

## field

In [42]:
# Przygotuj dane
df_to_insert = final_df[['id', 'name', 'formula', 'description', 'specialization', 'type', 'level', 'isActive']]
df_to_insert = df_to_insert.reset_index(drop=True)

# Połączenie z bazą
conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()
cursor.execute("DELETE FROM mainApp_field")

# 1. Wstaw nowe rekordy (ignoruj, jeśli id już istnieje)
cursor.executemany(
    """
    INSERT OR IGNORE INTO mainApp_field (id, name, formula, description, specialization, type, level, isActive)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """,
    df_to_insert.values.tolist()
)

# 2. Zaktualizuj description i specialization, jeśli są puste w istniejących rekordach
update_query = """
UPDATE mainApp_field
SET
    description = COALESCE(NULLIF(description, ''), ?),
    specialization = COALESCE(NULLIF(specialization, ''), ?),
    level = ?
WHERE id = ?
"""

# iteracja po wierszach final_df
for _, row in df_to_insert.iterrows():
    cursor.execute(update_query, (
        row['description'],
        row['specialization'],
        row['level'],
        row['id']
    ))

conn.commit()
conn.close()

## Course

In [43]:
df_to_insert = course_df.copy().dropna(axis=0)
df_to_insert = df_to_insert.reset_index(drop=True)  # reset indeksu, usunięcie starego
df_to_insert['id'] = range(1, len(df_to_insert)+1)       # nowe id od 0

# Zmień nazwy kolumn zgodnie z tabelą w bazie danych
df_to_insert = df_to_insert.rename(columns={
    'ects': 'ECTS',
})

# Wybierz tylko kolumny potrzebne do wstawienia
df_to_insert = df_to_insert[['id', 'name', 'ECTS', 'test_type', 'lecturer', 'description']]

In [44]:
# Połączenie z bazą danych
conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()
cursor.execute("DELETE FROM mainApp_course")
# INSERT OR IGNORE — wstawia tylko jeśli dany id nie istnieje
cursor.executemany(
    """
    INSERT OR IGNORE INTO mainApp_course (id, name, ECTS, test_type, lecturer, description)
    VALUES (?, ?, ?, ?, ?, ?)
    """,
    df_to_insert.values.tolist()
)

conn.commit()
conn.close()

## field_faculty

In [45]:
conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()
cursor.execute("DELETE FROM mainApp_field_faculty")

# Iteracja po wierszach field_faculty_df
for _, row in field_faculty_df.iterrows():
    field_id = row['field_id']
    faculty_id = row['faculty_id']

    # Pomijaj, jeśli faculty_id jest pustym stringiem
    if faculty_id == '':
        continue

    # Sprawdź, czy istnieje rekord z danym field_id
    cursor.execute(
        "SELECT id, faculty_id FROM mainApp_field_faculty WHERE field_id = ?",
        (field_id,)
    )
    existing = cursor.fetchone()

    if existing is None:
        # Nie istnieje — wstaw nowy rekord
        cursor.execute(
            """
            INSERT INTO mainApp_field_faculty (field_id, faculty_id)
            VALUES (?, ?)
            """,
            (field_id, faculty_id)
        )
    else:
        # Istnieje — aktualizuj, jeśli faculty_id jest różne
        existing_faculty_id = existing[1]
        if str(existing_faculty_id) != str(faculty_id):
            cursor.execute(
                """
                UPDATE mainApp_field_faculty
                SET faculty_id = ?
                WHERE field_id = ?
                """,
                (faculty_id, field_id)
            )

conn.commit()
conn.close()

## course_field

In [46]:
course_field_df = course_field_df.copy().reset_index(drop=True)
course_field_df['id'] = range(1, len(course_field_df) + 1)
course_field_df['course_id'] = course_field_df['course_id']+1

In [47]:
conn = sqlite3.connect('db.sqlite3')
cursor = conn.cursor()

# Wyczyść tabelę
cursor.execute("DELETE FROM mainApp_course_field")

# Wstaw dane z ręcznie ustawionym id
cursor.executemany(
    """
    INSERT INTO mainApp_course_field (id, course_id, field_id)
    VALUES (?, ?, ?)
    """,
    course_field_df[['id', 'course_id', 'field_id']].values.tolist()
)

conn.commit()
conn.close()